## 2.0_feature_engineering

by: Tom Goral

## OUTLINE

The purpose of this note book is to clean & prepare the anonymized data from  1.0_fetch_anonymize_ data.ipynb for application in a machine lerning model.

- [PROJECT OVERVIEW](#PROJECT-OVERVIEW)
- [PROBLEM STATEMENT](#PROBLEM-STATEMENT)

1. [STEP 1: LOAD DATA](#STEP-1:-LOAD-DATA)
2. [STEP 2: EXPLORE DATA](#STEP-2:-EXPLORE-DATA)
3. [STEP 3: VISUALIZE DATA](#STEP-3:-VISUALIZE-DATA)
4. [STEP 4: PREPROCESS DATA](#STEP-4:-PREPROCESS-DATA)
5. [STEP 5: EXPORT PREPROCESSED FEATURES & RESULT](#STEP-5:-EXPORT-PREPROCESSED-FEATURES-&-RESULT)

## PROJECT OVERVIEW
The USMCA (Canada, Mexico, US) industrial fastener market is significant.  It is valued \\$19B today and is expected to grow at a 3% CAGR reaching $22B by 2025 \[1] .  It’s a competitive market too with competent manufacturers and demanding customers that expect quick, competitive cost proposals for their custom requirements.  Today, custom fastener products require subject matter experts to accurately estimate the cost of each customer request.  This takes time with current tools and process. <br>


 As a product manager for a major fastener manufacturer, I am caught between timeliness and accuracy of cost proposals.  On the one hand, a timely cost estimate will please a customer.  On the other hand, an inaccurate cost estimate can frustrate a customer or backfire on the fastener manufacturer if the cost is too low!  A method to provide timely and accurate economic proposals is needed if our company wants to grow with this competitive market. 

\[1] Freedonia, "Global Industrial Fasteners," Freedonia, Cleveland, 2017. 


## PROBLEM STATEMENT
It takes a week from the receipt of a customer request to respond with an economic proposal.  Most of the time is used researching each feature cost, summing the individual costs and validating the total cost.  Although it is accurate, the time to deliver the economic proposal exceeds the three-day preference of most major customers.  Delivering an accurate economic proposal within three days would open more opportunities to win new business.<br>

One solution is to apply machine model regression to predict a cost using the features and costs of similar parts we make today. 

In [ ]:
# CUSTOM LIBRARIES
from utilities.xl2df         import xl2df
from utilities.hist_plot     import hist_plot
from utilities.print_metrics import print_metrics
from utilities.df2input      import df2input

# STANDARD LIBRARIES
import os
import sys
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category = UserWarning, module = "matplotlib")# Suppress matplotlib user warnings
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')  # Display inline matplotlib plots with IPython

from timeit import default_timer as timer
import datetime
now = datetime.datetime.now()

## STEP 1: LOAD DATA
Read in anonymized product data file.

In [ ]:
df = xl2df('data/anonymous.xlsx','anonymous',0)

In [ ]:
df.head()

In [ ]:
print(df.isnull().sum().sort_values(ascending=False))

In [ ]:
df.info()

In [ ]:
df.describe()

## STEP 2: EXPLORE DATA

* STATISTICS
    * ROWS: 55,140
    * COLS: 14
    * CELLS: 771,960
    * MAX NULLS:  SUB 8443<br><br>
* FOURTEEN FEATURES make up each record or part.<br><br>
* ELEVEN FEATURES are object columns describing each part.<br>
![](feature_list.jpg)<br>
    * The actual labels are kept confidential by applying generic labels through an "Anonymizer" program prior to this notebook.<br><br>
* THREE FEATURES are numerical
    * Column "qty" is an integer.  It defines the yearly consumption. 
    * Column "mm" is a float.  It defines the length of the part.  The "Nom" feature identified the cross sectional area.  Together they represent the parts volume or weight
    * Column "cost" is the current cost for each part at one point in time. <br><br>
* The index is set to the raw data index.  This will be useful to examine the confidential data when needed.

In [ ]:
x=df['qty']
num_bins = 2000
labels = {'title': 'HISTOGRAM of QUANTITY', 'ylabel': 'COUNT', 'xlabel': 'QUANTITY'}
xy_max_min = {'ymin': 0, 'ymax': 10000, 'xmin': 0 , 'xmax': 500000}
hist_plot(x, num_bins, labels, xy_max_min)

In [ ]:
x=df['mm']
num_bins = 40
labels = {'title': 'HISTOGRAM of LENGTH', 'ylabel': 'COUNT', 'xlabel': 'LENGTH'}
xy_max_min = {'ymin': 0, 'ymax': 10000, 'xmin': 0 , 'xmax': 500}
hist_plot(x, num_bins, labels, xy_max_min)

In [ ]:
x=df['cost']
num_bins = 100
labels = {'title': 'HISTOGRAM of COST', 'ylabel': 'COUNT', 'xlabel': 'COST'}
xy_max_min = {'ymin': 0, 'ymax': 10000, 'xmin': 0 , 'xmax':50 }
hist_plot(x, num_bins, labels, xy_max_min)

##  STEP 3: VISUALIZE DATA

Its a pretty safe bet that typcial quote requests will be for more than 10,000 parts of an average size that cost less than \\$1 each.  Examining the data frame describe and plotting the bins of "qty", "mm" and "cost" alerts me to outliers.  Although some of these may be legitimate records, I will eliminate them since there are 55,000 records and the file was not originally intended for my application.  I will have higher fidelity of data doing this.

## STEP 4: PREPROCESS DATA

Preproceseeing the data required the following.  First the features columns were identified and segmented into categorcial, numerical and ordinal groups.  The response was set to be the "cost".  From there all rows missing data were eliminated.  This dropped our record count by 13,248 from 55,140 to 41,892.  Outliers based on the numerical features were excluding leaving us with 7,965 records.  This is a large decline, but it assures us that we understand our data assumption. Lastly the categorical features were One-Hot-Encoded, OHE increasing our column count to 362.  The shape of our data went<br>

|   DATA      |       ROWS       |     COLS            |   CELLS   |
| :----------:| :--------------: | :-----------------: |:--------: |
| RAW         |      55,140      |    14               | 771,960   |
| PREPROCESSED|       7,965      |    362              | 2,883,330 |




### Segment  features from responses in the data

In [ ]:
categorical_features=['class', 'sub', 'assy', 'head', 'drive', 'thread',
                      'nom', 'point', 'heat', 'lock', 'plate'] 
            
numeric_features       = ["qty","mm"]               # floats or integers
ordinal_features       = []                         # unspecific scale high, medium, low
response_label         = "cost"

feature_columns        = categorical_features + numeric_features + ordinal_features
feature_columns.append(response_label)

features               = df[feature_columns]

print("                original features shape:",features.shape)
features               = df[feature_columns].dropna()                # eliminate rows with any empty features
print("eliminated empty records features shape:",features.shape)

###  Remove outliers
 
* Restrict the data to records where the numerical data can be associated with real parts 
    *  10,000 >= "qty" >= 100,000,000
    *  2 >=  "mm" >= 150 
    *  0 >  "cost" >= 1.00

In [ ]:
ranges = {'qty':[10000,100000000],'mm':[0,150], 'cost': [0, 1.0]}

records2delete = []
for key in ranges:
    minValue = ranges[key][0]
    maxValue = ranges[key][1]
    a = (features[features[key] <minValue].index).tolist()
    b = (features[features[key] >maxValue].index).tolist()
    c = a+b
    records2delete.extend(c)
    
features.drop(index=records2delete, inplace = True)
print("    Eliminate Outside Range Features shape:",features.shape)

### Drop the response column & OHE categorical features

In [ ]:
response = features[response_label]
features.drop([response_label],axis=1, inplace =True)


ohe_columns            = categorical_features + ordinal_features
features               = pd.get_dummies(features, columns=ohe_columns)   # OHE categorical features
print("OHE features shape:",features.shape)

##  STEP 5: EXPORT PREPROCESSED FEATURES & RESULT

In [ ]:
features.to_excel("features.xlsx",sheet_name='features')
response.to_excel("response.xlsx",sheet_name='response')

##  GO TO:   3.0_train_deploy_model.ipynb